<a href="https://colab.research.google.com/github/ajou-y0uC4N/y0uC4N/blob/main/%EB%B9%85%EC%9D%91%EB%B3%B4_case10_AE_IF_LOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:

data = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type10.csv')#.to_numpy()


X = data[data['type'] == 'Normal']
y = X['type']
y = y.map({'Normal': 0, 'Attack': 1}).to_numpy()
X = X.drop(columns =['type'])
X['messageID'] = X['messageID'].astype('category')
X = X.drop(columns =['isCan0','isService','serviceNODEID','time'])
X = X.drop(columns = ['data[0]','data[7]'])
print(X.shape)


(131530, 11)


In [22]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
input_dim = X_scaled.shape[1]
encoding_dim = 6

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Autoencoder 학습
autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/30
3289/3289 [==============================] - 9s 2ms/step - loss: 0.7185 - val_loss: 0.6411
Epoch 2/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.6081 - val_loss: 0.6226
Epoch 3/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.5995 - val_loss: 0.6210
Epoch 4/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.5972 - val_loss: 0.6204
Epoch 5/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.5963 - val_loss: 0.6202
Epoch 6/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.5958 - val_loss: 0.6200
Epoch 7/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.5955 - val_loss: 0.6197
Epoch 8/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.5953 - val_loss: 0.6195
Epoch 9/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.5952 - val_loss: 0.6193
Epoch 10/30
3289/3289 [==============================] - 8s 2ms/step - loss: 0.595

In [23]:
# 데이터를 인코딩
X_encoded = autoencoder.predict(X_scaled)

4111/4111 [==============================] - 5s 1ms/step


In [25]:
#data.loc[data['type']=='Normal', 'type'] = 0
#data.loc[data['type']=='Attack', 'type'] = 1

fullDataX1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type10.csv')#.to_numpy()

#X['messageID'] = data['messageID']
fX1 = fullDataX1.drop(columns =['type'])
fX1['messageID'] = fX1['messageID'].astype('category')
fX1 = fX1.drop(columns =['isCan0','isService','serviceNODEID','time'])
fX1 = fX1.drop(columns = ['data[0]','data[7]'])
print(fX1.shape)
fX1_scaled = scaler.fit_transform(fX1)
fX1_encoded = autoencoder.predict(fX1_scaled)

(207380, 11)
6481/6481 [==============================] - 8s 1ms/step


In [26]:
if_model = IsolationForest(contamination=0.14)  # contamination 값 조정 가능
if_model.fit(X_encoded)

# 이상치 예측
if_predictions = if_model.predict(fX1_encoded)

In [27]:
from sklearn.metrics import confusion_matrix, classification_report
fy = fullDataX1['type']
fy = fy.map({'Normal': 1, 'Attack': 0}).to_numpy()
if_predictions = np.where(if_predictions == -1, 0, if_predictions)
print(np.unique(if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy, if_predictions)
class_report = classification_report(fy, if_predictions)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([ 64347, 143033]))
Confusion Matrix:
[[ 41847  34003]
 [ 22500 109030]]

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.55      0.60     75850
           1       0.76      0.83      0.79    131530

    accuracy                           0.73    207380
   macro avg       0.71      0.69      0.70    207380
weighted avg       0.72      0.73      0.72    207380



In [11]:
# 모델 저장
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case10_if_model.pkl', 'wb') as file:
    pickle.dump(if_model, file)

In [12]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case10_if_model.pkl', 'rb') as file:
    loaded_if_model = pickle.load(file)

# 불러온 모델로 예측
fy1 = fullDataX1['type']
fy1 = fy1.map({'Normal': 1, 'Attack': 0}).to_numpy()
loaded_if_predictions = loaded_if_model.predict(fX1_encoded)
loaded_if_predictions = np.where(loaded_if_predictions == -1, 0, loaded_if_predictions)
print(np.unique(loaded_if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy1, loaded_if_predictions)
class_report = classification_report(fy1, loaded_if_predictions)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([106901, 134420]))
Confusion Matrix:
[[34873 46215]
 [72028 88205]]

Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.43      0.37     81088
           1       0.66      0.55      0.60    160233

    accuracy                           0.51    241321
   macro avg       0.49      0.49      0.48    241321
weighted avg       0.55      0.51      0.52    241321



In [10]:
# Local Outlier Factor 모델 정의 및 학습
lof_model = LocalOutlierFactor(n_neighbors=500, contamination=0.5, novelty=True)  # n_neighbors 및 contamination 값 조정 가능
lof_model.fit(X_encoded)
lof_predictions = lof_model.predict(fX1_encoded)
# 이상치라면 -1, 정상치라면 1로 표시됨


In [11]:
fy = fullDataX1['type']
fy = fy.map({'Normal': 1, 'Attack': 0}).to_numpy()
lof_predictions = np.where(lof_predictions == -1, 0, lof_predictions)
print(np.unique(lof_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy, lof_predictions)
class_report = classification_report(fy, lof_predictions)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([235525,   5796]))
Confusion Matrix:
[[ 80708    380]
 [154817   5416]]

Classification Report:
              precision    recall  f1-score   support

           0       0.34      1.00      0.51     81088
           1       0.93      0.03      0.07    160233

    accuracy                           0.36    241321
   macro avg       0.64      0.51      0.29    241321
weighted avg       0.74      0.36      0.21    241321



In [12]:
# 모델 저장
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_lof_model.pkl', 'wb') as file:
    #pickle.dump(lof_model, file)

In [ ]:
#with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_lof_model.pkl', 'rb') as file:
    #loaded_lof_model = pickle.load(file)

# 불러온 모델로 예측
#loaded_lof_predictions = loaded_lof_model.predict(fX1_encoded)
#print(np.unique(loaded_lof_predictions, return_counts=True))